# Tutorial 2: Integrate DLPFC data with MaskGraphene

In this tutorial, we show how to use MG to integrate DLPFC data. As an example, we analyze the 151507/151508 sample pair of the dorsolateral prefrontal cortex (DLPFC) dataset. 

We acquired the data from the spatialLIBD webpage, including manual annotations. Before running the model, please download the input data via [zenodo link]()

### Loading packages

In [3]:
import logging
import numpy as np
from tqdm import tqdm
import torch
import pickle
import wandb

from utils import (
    build_args_ST,
    create_optimizer,
    set_random_seed,
    get_current_lr,
)
from datasets.data_proc import load_ST_dataset_hard, load_ST_dataset_hard_erase
from datasets.st_loading_utils import create_dictionary_otn, visualization_umap_spatial, create_dictionary_mnn, cal_layer_based_alignment_result, cal_layer_based_alignment_result_mhypo
from models import build_model_ST
import os
import scanpy as sc
import sklearn.metrics.pairwise

from maskgraphene_main import main


### HP setup

In [5]:
args = build_args_ST()
args.max_epoch=2000
args.max_epoch_triplet=500
args.section_ids="151507,151508"
args.num_class=7
args.num_hidden="512,32"
args.alpha_l=1
args.lam=1 
args.loss_fn="sce" 
args.mask_rate=0.5 
args.in_drop=0 
args.attn_drop=0 
args.remask_rate=0.1
args.seeds=[2023] 
args.num_remasking=1 
args.hvgs=3000 
args.dataset="DLPFC" 
args.consecutive_prior=1 
args.lr=0.001

#### remember to change these paths to your data path/link path
args.st_data_dir="/home/yunfei/spatial_benchmarking/benchmarking_data/DLPFC12"
args.pi_dir="/home/yunfei/spatial_dl_integration/MaskGraphene/PI"

### MG training

In [ ]:
out = main(args)